In [ ]:
!pip install dominate

In [ ]:
!pip install visdom


In [ ]:

!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
%cd pytorch-CycleGAN-and-pix2pix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Training

In [ ]:
!python /content/pytorch-CycleGAN-and-pix2pix/train.py --dataroot /content/drive/MyDrive/ --name scratch_restoration_experiment --model pix2pix --direction AtoB --dataset_mode aligned --gpu_ids 0 --display_id -1 --checkpoints_dir /content/drive/MyDrive/checkpoints_2/ --batch_size 2 --continue_train --epoch_count 189


# Testing

In [ ]:
!python /content/pytorch-CycleGAN-and-pix2pix/test.py --gpu_ids 0 --dataroot /content/drive/MyDrive/ --name scratch_restoration_experiment --model pix2pix --direction AtoB --dataset_mode aligned   --results_dir /content/drive/MyDrive/results_1/ --checkpoints_dir /content/drive/MyDrive/checkpoints/ --phase test --num_test 238


# Single Image Inference

In [ ]:
!python /content/pytorch-CycleGAN-and-pix2pix/test.py --dataroot /content/data/test --name scratch_restoration_experiment --model pix2pix --gpu_ids 0 --checkpoints_dir /content/drive/MyDrive/checkpoints/ --results_dir /content/drive/MyDrive/results_testing_test --phase test --eval --batch_size 1 --dataset_mode single --netG unet_256 --no_flip


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Paths to your images
fake_folder_path = '/content/drive/MyDrive/results_testing_2/scratch_restoration_experiment/test_latest/images'
real_folder_path = '/content/drive/MyDrive/results_testing_2/scratch_restoration_experiment/test_latest/images'

# Get lists of image files
fake_images = sorted([f for f in os.listdir(fake_folder_path) if f.startswith('fake_B_test_')])
real_images = sorted([f for f in os.listdir(real_folder_path) if f.startswith('real_A_test_')])

# Ensure both folders have the same number of images
assert len(fake_images) == len(real_images), "Mismatch in number of images"

# Set up the figure for displaying images
num_images = len(fake_images)
fig, axes = plt.subplots(num_images, 2, figsize=(12, 6 * num_images))  # Adjust figure size based on number of images

# Loop through each image pair
for i in range(num_images):
    # Load images
    fake_image = Image.open(os.path.join(fake_folder_path, fake_images[i])).convert('RGB')
    real_image = Image.open(os.path.join(real_folder_path, real_images[i])).convert('RGB')

    # Display real image
    axes[i, 0].imshow(real_image)
    axes[i, 0].axis('off')  # Hide axes
    axes[i, 0].set_title(f'Real Image {i + 1}')

    # Display fake image
    axes[i, 1].imshow(fake_image)
    axes[i, 1].axis('off')  # Hide axes
    axes[i, 1].set_title(f'Fake Image {i + 1}')

# Adjust layout
plt.tight_layout()
# Show the images
plt.show()


# Evaluation

###PSNR (Peak Signal-to-Noise Ratio)

In [ ]:
import os
import cv2
import numpy as np

# Function to calculate PSNR
def calculate_psnr(real_image, fake_image):
    mse = np.mean((real_image - fake_image) ** 2)
    if mse == 0:
        return 100  # Perfect match
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

# Path to the directory containing images
image_folder = "/content/drive/MyDrive/results_2/scratch_restoration_experiment/test_latest/images"

# Lists to store PSNR values
psnr_values = []

# Loop over the images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith("_real_B.png"):
        # Get the corresponding fake image name
        fake_filename = filename.replace("_real_B.png", "_fake_B.png")

        real_image_path = os.path.join(image_folder, filename)
        fake_image_path = os.path.join(image_folder, fake_filename)

        # Check if the corresponding fake image exists
        if os.path.exists(fake_image_path):
            # Read the real and fake images
            real_image = cv2.imread(real_image_path, cv2.IMREAD_COLOR)
            fake_image = cv2.imread(fake_image_path, cv2.IMREAD_COLOR)

            # Ensure both images are the same size
            if real_image.shape == fake_image.shape:
                # Calculate PSNR
                psnr = calculate_psnr(real_image, fake_image)
                psnr_values.append(psnr)
            else:
                print(f"Image size mismatch between {filename} and {fake_filename}")

# Calculate the average PSNR
if psnr_values:
    average_psnr = np.mean(psnr_values)
    print(f"Average PSNR for all images: {average_psnr}")
else:
    print("No valid image pairs found.")


Average PSNR for all images: 30.03188614241356


###SSIM (Structural Similarity Index)

In [ ]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(img1, img2):
    # Convert images to grayscale for SSIM calculation (optional)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return ssim(img1_gray, img2_gray)

def evaluate_ssim(folder_path):
    total_ssim = 0.0
    num_images = 0

    # Loop through the folder to get image pairs
    for filename in os.listdir(folder_path):
        if 'fake_B' in filename:  # Look for the fake_B image
            fake_image_path = os.path.join(folder_path, filename)
            real_image_path = fake_image_path.replace('fake_B', 'real_B')  # Find the corresponding real_B image

            if os.path.exists(real_image_path):
                # Load the images
                fake_img = cv2.imread(fake_image_path)
                real_img = cv2.imread(real_image_path)

                # Compute SSIM for the image pair
                score = calculate_ssim(real_img, fake_img)
                total_ssim += score
                num_images += 1

    if num_images == 0:
        print("No valid image pairs found.")
        return 0

    # Calculate the average SSIM
    average_ssim = total_ssim / num_images
    print(f'Average SSIM: {average_ssim}')
    return average_ssim

# Set the folder path where the images are stored
folder_path = '/content/drive/MyDrive/results_2/scratch_restoration_experiment/test_latest/images'
evaluate_ssim(folder_path)


Average SSIM: 0.8583425579409666


0.8583425579409666